The goal of this notebook is to test out morphologocal operations to isolate the quadrat. They're pretty finicky so I need to know how they work on all the test photos.

### Setup

In [1]:
from functions import reorder_file_paths
import cv2
from skimage import img_as_float, img_as_ubyte
from matplotlib import pyplot as plt
from functions import apply_otsu
from skimage.filters.rank import entropy
from skimage.morphology import disk, isotropic_closing, binary_closing
from scipy import ndimage
import numpy as np

In [2]:
# Determine the folder of images that will be looped through
path = 'test_photos/*'

# Get list of files for looping through
filenames = reorder_file_paths(path)

print(filenames)

['test_photos\\image_0.jpg', 'test_photos\\image_1.jpg', 'test_photos\\image_2.jpg', 'test_photos\\image_3.jpg', 'test_photos\\image_4.jpg', 'test_photos\\image_5.jpg', 'test_photos\\image_6.JPG', 'test_photos\\image_7.JPG', 'test_photos\\image_8.JPG', 'test_photos\\image_9.JPG', 'test_photos\\image_10.JPG', 'test_photos\\image_11.jpg', 'test_photos\\image_12.jpg', 'test_photos\\image_13.jpg', 'test_photos\\image_14.jpg', 'test_photos\\image_15.jpg', 'test_photos\\image_16.jpg', 'test_photos\\image_17.jpg', 'test_photos\\image_18.JPG', 'test_photos\\image_19.JPG', 'test_photos\\image_20.jpg', 'test_photos\\image_21.jpg', 'test_photos\\image_22.jpg', 'test_photos\\image_23.jpg', 'test_photos\\image_24.jpg']


In [3]:
greenhouse_test_filenames = ['test_photos\\image_6.JPG', 'test_photos\\image_7.JPG', 'test_photos\\image_8.JPG', 'test_photos\\image_9.JPG', 'test_photos\\image_10.JPG']

### Use threshold to pick up on presence of quadrat

In [16]:
for i, filename in enumerate(filenames):
    # Skip over the greenhouse images
    if filename in greenhouse_test_filenames:
        pass

    else:
        # Load in the images in color
        img = cv2.imread(filename, 1)

        # Get a grayscale version of the image
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Convert images into floats
        img_float = img_as_float(gray)

        # If image is <= to the threshold number, it's True. False otherwise.
        img_binary =  (img_float <= .9) # .90 is meant to pick up the white of the quadrat.

        # Convert back to ubyte
        img_ubyte = img_as_ubyte(img_binary)

        # Denoise the image to get rid of the grass that made it through
        img_median = ndimage.median_filter(img_ubyte, 9)

        # Define a kernel
        kernel = np.ones((3,3), np.uint8)

        # Dilate
        img_eroded = cv2.erode(img_median, kernel, iterations = 3)

        # Invert the image 
        img_eroded = cv2.bitwise_not(img_eroded)

        # Detect edges using Canny
        edges = cv2.Canny(img_eroded, 50, 150, apertureSize=7)

        # Detect lines using Hough Line Transform
        lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 100, minLineLength=50, maxLineGap=100)

        # Draw the lines
        if lines is not None:
            for line in lines:
                x1, y1, x2, y2 = line[0]
                cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 2)

        # Find contours
        contours, _ = cv2.findContours(img_eroded, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            # Approximate the contour to a polygon
            epsilon = 0.02 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)
            
            # If the approximated polygon has 4 points, we assume it is a rectangle
            if len(approx) == 4:
                cv2.drawContours(img, [approx], 0, (0, 0, 0), 5)
                for point in approx:
                    cv2.circle(img, tuple(point[0]), 5, (0, 0, 0), -1)


        # Define a kernel
        # kernel = np.ones((3,3), np.uint8)

        # # Dilate
        # img_eroded = cv2.erode(img_median, kernel, iterations = 3)
        # img_er_and_dilated = cv2.dilate(img_eroded, kernel, iterations = 1)

        # Use binary closing to potentially close gaps
        # mask = binary_closing(img_eroded, 2)

        cv2.imwrite(f'results/isolating_quadrat/image_{i}_edge_detected.jpg', img)

        print(f'Saved {filename}')

        # Display the images
    #     cv2.imshow(f'img {i}', img_ubyte)

# cv2.waitKey(0)
# cv2.destroyAllWindows()

Saved test_photos\image_0.jpg
Saved test_photos\image_1.jpg
Saved test_photos\image_2.jpg
Saved test_photos\image_3.jpg
Saved test_photos\image_4.jpg
Saved test_photos\image_5.jpg
Saved test_photos\image_11.jpg
Saved test_photos\image_12.jpg
Saved test_photos\image_13.jpg
Saved test_photos\image_14.jpg
Saved test_photos\image_15.jpg
Saved test_photos\image_16.jpg
Saved test_photos\image_17.jpg
Saved test_photos\image_18.JPG
Saved test_photos\image_19.JPG
Saved test_photos\image_20.jpg
Saved test_photos\image_21.jpg
Saved test_photos\image_22.jpg
Saved test_photos\image_23.jpg
Saved test_photos\image_24.jpg
